## ResNet20 (not Class-based)

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

nb_classes = 10
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.cifar10.load_data()
train_x, test_x = train_x / 255.0, test_x / 255.0

train_y = tf.keras.utils.to_categorical(train_y, nb_classes)
test_y = tf.keras.utils.to_categorical(test_y, nb_classes)

In [ ]:
%%time
kernel_initializer = 'he_normal'
def resnet_block(inputs, f1, f2, s):
    # residual part
    residual_layers = keras.Sequential([
      layers.Conv2D(filters=f1, kernel_size=3, strides=s, padding='same', kernel_initializer=kernel_initializer, use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU(),
      layers.Conv2D(filters=f2, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer, use_bias=False),
      layers.BatchNormalization()
    ])
    residual_output = residual_layers(inputs)

    # shortcut part
    if s == 2:
      shortcut_layers = keras.Sequential([
        layers.Conv2D(filters=f2, kernel_size=1, strides=s, padding='same', kernel_initializer=kernel_initializer, use_bias=False),
        layers.BatchNormalization()
      ])
      shortcut = shortcut_layers(inputs)
    else:
      shortcut = inputs
    
    # add residual and shortcut
    x = keras.layers.add([residual_output, shortcut]) # Or, x = residual_output + shortcut
    output = keras.layers.ReLU()(x) # Or, x = tf.nn.relu(x)
    
    return output

def resnet20(inputs):
    # stem layers
    first_layers = keras.Sequential([
      layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer, use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU()
    ])
    x = first_layers(inputs)

    # residual blocks
    x = resnet_block(x, 16, 16, 1)
    x = resnet_block(x, 16, 16, 1)
    x = resnet_block(x, 16, 16, 1)

    x = resnet_block(x, 32, 32, 2)
    x = resnet_block(x, 32, 32, 1)
    x = resnet_block(x, 32, 32, 1)

    x = resnet_block(x, 64, 64, 2)
    x = resnet_block(x, 64, 64, 1)
    x = resnet_block(x, 64, 64, 1)

    # head layers
    last_layers = keras.Sequential([
      layers.GlobalAveragePooling2D(),
      layers.Flatten(),
      layers.Dense(10, kernel_initializer=kernel_initializer)
    ])
    output = last_layers(x)

    return output

# build keras.Model
inputs = keras.Input(shape=(32, 32, 3))
outputs = resnet20(inputs)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.train.AdamOptimizer(0.005),
              metrics=['accuracy'])
model.summary()
epoch = 7
batch_size = 128
model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epoch, validation_data=(test_x, test_y))